# Amazon Bedrock에서 채팅 완성 클라이언트를 사용하는 방법

EasyLLM은 `gpt-3.5-turbo` 및 `gpt-4`를 Amazon Bedrock 모델로 대체하는 추상화 계층으로 사용할 수 있습니다.

클라이언트만 변경하여 OpenAI API에서 자체 애플리케이션을 변경할 수 있습니다.

채팅 모델은 일련의 메시지를 입력으로 사용하고 AI가 작성한 메시지를 출력으로 반환합니다.

이 가이드에서는 몇 가지 예제 API 호출을 통해 채팅 형식을 보여줍니다.

## 0. 설정

Amazon Bedrock에서 `easyllm`을 사용하려면 먼저 모델에 대한 권한 및 액세스를 설정해야 합니다. 아래 지침에 따라 이 작업을 수행할 수 있습니다.
* https://docs.aws.amazon.com/IAM/latest/UserGuide/getting-set-up.html
* https://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html
* https://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html

### 1. easyllm 라이브러리 가져오기

In [ ]:
# 필요한 경우 EasyLLM Python 라이브러리의 최신 버전으로 설치 및/또는 업그레이드합니다.
%pip install --upgrade easyllm[bedrock] 

In [6]:
# EasyLLM API를 호출하기 위한 EasyLLM Python 라이브러리를 가져옵니다.
import easyllm

### 2. 채팅 API 호출 예제

채팅 API 호출에는 두 가지 필수 입력이 있습니다.
- `model`: 사용하려는 모델의 이름(예: `huggingface-pytorch-tgi-inference-2023-08-08-14-15-52-703`) 또는 API만 호출하려면 비워 둡니다.
- `messages`: 메시지 개체 목록이며 각 개체에는 두 가지 필수 필드가 있습니다.
    - `role`: 메신저의 역할( `system`, `user` 또는 `assistant`)
    - `content`: 메시지 내용(예: `아름다운 시를 써주세요`)

OpenAI API와 비교하여 `huggingface` 모듈은 프롬프트 및 중지 시퀀스를 사용자 지정하는 데 사용할 수 있는 `prompt_builder` 및 `stop_sequences` 매개변수도 노출합니다. EasyLLM 패키지에는 프롬프트 빌더 유틸리티가 함께 제공됩니다.

채팅 형식이 실제로 어떻게 작동하는지 확인하기 위해 예제 채팅 API 호출을 살펴보겠습니다.

In [2]:
import os 
# 프롬프트 빌더용 환경 변수 설정
os.environ["BEDROCK_PROMPT"] = "anthropic" # vicuna, wizardlm, stablebeluga, open_assistant
os.environ["AWS_REGION"] = "us-east-1"  # 해당 지역으로 변경
# os.environ["AWS_ACCESS_KEY_ID"] = "XXX" # boto3 세션을 사용하지 않는 경우 필요
# os.environ["AWS_SECRET_ACCESS_KEY"] = "XXX" # boto3 세션을 사용하지 않는 경우 필요

from easyllm.clients import bedrock

response = bedrock.ChatCompletion.create(
    model="anthropic.claude-v2",
    messages=[
        {"role": "user", "content": "2 + 2는 무엇인가요?"},
    ],
      temperature=0.9,
      top_p=0.6,
      max_tokens=1024,
      debug=False,
)
response



{'completion': ' 2 + 2 = 4', 'stop_reason': 'stop_sequence'}


{'id': 'hf-Mf7UqliZQP',
 'object': 'chat.completion',
 'created': 1698333425,
 'model': 'anthropic.claude-v2',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': '2 + 2 = 4'},
   'finish_reason': 'stop_sequence'}],
 'usage': {'prompt_tokens': 9, 'completion_tokens': 9, 'total_tokens': 18}}

보시다시피 응답 개체에는 몇 가지 필드가 있습니다.
- `id`: 요청 ID
- `object`: 반환된 개체 유형(예: `chat.completion`)
- `created`: 요청 타임스탬프
- `model`: 응답을 생성하는 데 사용된 모델의 전체 이름
- `usage`: 프롬프트, 완성 및 합계를 계산하여 회신을 생성하는 데 사용된 토큰 수
- `choices`: 완성 개체 목록(`n`을 1보다 크게 설정하지 않은 경우 하나만)
    - `message`: 모델에서 생성한 메시지 개체( `role` 및 `content` 포함)
    - `finish_reason`: 모델이 텍스트 생성을 중지한 이유(`stop` 또는 `max_tokens` 제한에 도달한 경우 `length`)
    - `index`: 선택 항목 목록에서 완성의 인덱스

다음을 사용하여 회신만 추출합니다.

In [3]:
print(response['choices'][0]['message']['content'])

2 + 2 = 4


대화 기반이 아닌 작업도 첫 번째 사용자 메시지에 지침을 배치하여 채팅 형식에 맞출 수 있습니다.

예를 들어 모델에게 해적 블랙비어드 스타일로 비동기 프로그래밍을 설명하도록 요청하려면 다음과 같이 대화를 구성할 수 있습니다.

In [4]:
# 시스템 메시지가 있는 예제
response = bedrock.ChatCompletion.create(
    model="anthropic.claude-v2",
    messages=[
        {"role": "system", "content": "당신은 도움이 되는 조수입니다."},
        {"role": "user", "content": "수학 교사 스타일로 비동기 프로그래밍을 설명해주세요."},
    ],
)

print(response['choices'][0]['message']['content'])


{'completion': ' Okay class, today we\'re going to learn about asynchronous programming. Asynchronous means things happening at different times, not necessarily in order. It\'s like when you\'re cooking dinner - you might put the pasta on to boil, then start chopping vegetables while the pasta cooks. You don\'t have to wait for the pasta to finish boiling before you can start on the vegetables. The two tasks are happening asynchronously. \n\nIn programming, asynchronous functions allow the code to execute other operations while waiting for a long-running task to complete. Let\'s look at an example:\n\n```js\nfunction cookPasta() {\n  console.log("Putting pasta on to boil...");\n  // Simulate a long task\n  setTimeout(() => {\n    console.log("Pasta done!");\n  }, 5000); \n}\n\nfunction chopVegetables() {\n  console.log("Chopping vegetables...");\n}\n\ncookPasta();\nchopVegetables();\n```\n\nWhen we call `cookPasta()`, it starts the timer but doesn\'t wait 5 seconds - it immediately mov

In [5]:
# 시스템 메시지가 없고 디버그 플래그가 켜진 예제:
response = bedrock.ChatCompletion.create(
    model="anthropic.claude-v2",
    messages=[
        {"role": "user", "content": "해적 블랙비어드 스타일로 비동기 프로그래밍을 설명해주세요."},
    ]
)

print(response['choices'][0]['message']['content'])


{'completion': " Aye matey! Asynchronous programming be when ye fire yer cannons without waiting fer each shot to hit. Ye keep loadin' and shootin' while the cannonballs sail through the air. Ye don't know exactly when they'll strike the target, but ye keep sendin' 'em off. \n\nThe ship keeps movin' forward, not stalled waiting fer each blast. Other pirates keep swabbin' the decks and hoistin' the sails so we make progress while the cannons thunder. We tie callbacks to the cannons to handle the boom when they finally hit.\n\nArrr! Asynchronous programmin' means ye do lots o' tasks at once, not blocked by waitin' fer each one to finish. Ye move ahead and let functions handle the results when ready. It be faster than linear code that stops at each step. Thar be treasures ahead, lads! Keep those cannons roarin'!", 'stop_reason': 'stop_sequence'}
아이, 친구! 비동기 프로그래밍은 각 포탄이 명중하기를 기다리지 않고 대포를 발사하는 것과 같네. 포탄이 공중을 날아가는 동안 계속 장전하고 발사하는 거지. 언제 목표물에 명중할지는 정확히 모르지만 계속 발사하는 거야. \n\n배는 각 폭발을 기다리며 멈추지 

### 3. 소수 샷 프롬프트

경우에 따라 원하는 것을 모델에게 말하는 것보다 보여주는 것이 더 쉽습니다.

원하는 것을 모델에게 보여주는 한 가지 방법은 가짜 예제 메시지를 사용하는 것입니다.

예를 들어:

In [6]:
# 비즈니스 전문 용어를 더 간단한 말로 번역하도록 모델을 준비시키는 가짜 소수 샷 대화 예제
response = bedrock.ChatCompletion.create(
    model="anthropic.claude-v2",
    messages=[
        {"role": "system", "content": "당신은 도움이 되고 패턴을 따르는 조수입니다."},
        {"role": "user", "content": "다음 기업 전문 용어를 평이한 영어로 번역하는 것을 도와주세요."},
        {"role": "assistant", "content": "물론입니다, 기꺼이 도와드리겠습니다!"},
        {"role": "user", "content": "새로운 시너지가 최고 수익 성장을 이끌 것입니다."},
        {"role": "assistant", "content": "함께 잘 작동하는 것들이 수익을 증가시킬 것입니다."},
        {"role": "user", "content": "증가된 레버리지 기회에 대해 논의할 시간이 더 있을 때 다시 논의합시다."},
        {"role": "assistant", "content": "더 잘하는 방법에 대해 덜 바쁠 때 나중에 이야기합시다."},
        {"role": "user", "content": "이 늦은 전환은 고객 결과물에 대해 바다를 끓일 시간이 없다는 것을 의미합니다."},
    ],
)

print(response["choices"][0]["message"]["content"])


{'completion': " Changing direction at the last minute means we don't have time to do an exhaustive analysis for what we're providing to the client.", 'stop_reason': 'stop_sequence'}
마지막 순간에 방향을 바꾸면 고객에게 제공하는 것에 대해 철저한 분석을 할 시간이 없습니다.


대화 엔지니어링에 대한 모든 시도가 처음부터 성공하는 것은 아닙니다.

첫 번째 시도가 실패하면 모델을 준비하거나 조정하는 다양한 방법을 실험하는 것을 두려워하지 마십시오.

예를 들어, 한 개발자는 모델이 더 높은 품질의 응답을 제공하도록 조정하는 데 도움이 되도록 "지금까지 훌륭했습니다. 완벽했습니다."라는 사용자 메시지를 삽입했을 때 정확도가 증가하는 것을 발견했습니다.

모델의 신뢰성을 높이는 방법에 대한 더 많은 아이디어는 [신뢰성 향상 기술](../techniques_to_improve_reliability.md)에 대한 가이드를 참조하십시오. 비채팅 모델용으로 작성되었지만 많은 원칙이 여전히 적용됩니다.